In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Uploading CPS data

In [2]:
cps = pd.read_csv('cps.csv')

# Best Linear Predictor

In [3]:
ols1 = smf.ols(formula='log_wage ~ educ',data=cps).fit(cov_type='HC1')

print(ols1.summary())

                            OLS Regression Results                            
Dep. Variable:               log_wage   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     2441.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        12:10:17   Log-Likelihood:                -29890.
No. Observations:               22715   AIC:                         5.978e+04
Df Residuals:                   22713   BIC:                         5.980e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.5711      0.034    254.650      0.0

In [4]:
beta1 = ols1._results.params[1]

In [5]:
ols2 = smf.ols(formula='log_wage ~ educ+age',data=cps).fit(cov_type='HC1')

print(ols2.summary())

                            OLS Regression Results                            
Dep. Variable:               log_wage   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     2684.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        12:10:17   Log-Likelihood:                -28323.
No. Observations:               22715   AIC:                         5.665e+04
Df Residuals:                   22712   BIC:                         5.668e+04
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.4145      0.039    187.891      0.0

In [6]:
beta2_0 = ols2._results.params[0]
beta2_1 = ols2._results.params[1]
beta2_2 = ols2._results.params[2]

# Creating Interval Data

In [7]:
# def createIntervalData(df, Y, thresholds):
#     # The function accepts a dataframe, df, and make interval data from the Y (string) variable. 
#     # It adds to the dataframe the lower and upper values for Y (based on the thresholds) and the covariates.
    
#     thresholds = np.array(thresholds)

#     idx = [sum(t <= y for t in thresholds)-1 for y in df[Y]]
#     df[Y+'_l'] = thresholds[idx]
#     df[Y+'_u'] = thresholds[np.array(idx)+1]
    
#     df['log'+Y+'_l']= np.log(df[Y+'_l'])
#     df['log'+Y+'_u']= np.log(df[Y+'_u'])
    
#     return df

In [8]:
# wage_quantiles = np.array(cps['wage'].quantile([0.0, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]))
# wage_quantiles[-1]+=1
# logwage_quantiles = np.log(wage_quantiles)

# cps['wage_qt'] = np.floor(cps['wage'].rank(method = 'max', pct=True)*10).astype(int).replace({10:9})
# cps['wage_lower'] = wage_quantiles[cps['wage_qt']]
# cps['wage_upper'] = wage_quantiles[cps['wage_qt']+1]

# cps['logwage_lower'] = logwage_quantiles[cps['wage_qt']]
# cps['logwage_upper'] = logwage_quantiles[cps['wage_qt']+1]

In [9]:
# wage_quantiles = np.array(cps['wage'].quantile(np.linspace(0,1,11)))
# wage_quantiles[-1]+=1
# cps = createIntervalData(cps, 'wage', wage_quantiles)

In [10]:
cps

,Unnamed: 0.1,Unnamed: 0,age,wage,educ,log_wage,wage_l,wage_u,logwage_l,logwage_u
0,0,0,22,12000,13,9.392662,9000.0,15000.0,9.104980,9.615805
1,1,1,21,3500,13,8.160518,1.0,9000.0,0.000000,9.104980
2,2,3,49,30000,13,10.308953,30000.0,36000.0,10.308953,10.491274
3,3,4,31,32000,16,10.373491,30000.0,36000.0,10.308953,10.491274
4,4,5,42,89630,21,11.403445,71000.0,362303.0,11.170435,12.800236
...,...,...,...,...,...,...,...,...,...,...
22710,22710,26111,37,32000,14,10.373491,30000.0,36000.0,10.308953,10.491274
22711,22711,26112,50,56000,12,10.933107,53000.0,71000.0,10.878047,11.170435
22712,22712,26113,24,9000,12,9.104980,9000.0,15000.0,9.104980,9.615805
22713,22713,26114,49,29000,12,10.275051,25000.0,30000.0,10.126631,10.308953


In [11]:
# cps.to_csv("cps.csv")

# Partial Identification

In [12]:
import setBLP

In [13]:
# Preparing vector and matrix versions of the data

yl = cps.logwage_l;
yu = cps.logwage_u;
x1 = cps.age;
x2 = cps.educ;
x = cps[['age','educ']];

## Testing

In [14]:
r1=setBLP.oneDproj(yl,yu,x1)

In [15]:
vars(r1)

{'bound': [-0.004887858329798024, 0.14865963017399167],
 'Htest': <setBLP.TestResults at 0x150d1bfd0>,
 'dHtest': <setBLP.TestResults at 0x150d36a50>}

In [16]:
vars(r1.Htest)

{'TestStat': None,
 'criticalVal': 0.7047452080972225,
 'ConfidenceInterval': [-0.00956387188006217, 0.15333564372425582]}

In [17]:
vars(r1.dHtest)

{'TestStat': None,
 'criticalVal': 0.5974789676538764,
 'ConfidenceInterval': [-0.008852155946365771, 0.15262392779055942]}

In [18]:
r2=setBLP.oneDproj(yl,yu,x2)

In [19]:
vars(r2)

{'bound': [0.0017860422219384202, 0.3685793555817098],
 'Htest': <setBLP.TestResults at 0x150d372d0>,
 'dHtest': <setBLP.TestResults at 0x150d35d50>}

In [20]:
vars(r2.Htest)

{'TestStat': None,
 'criticalVal': 2.5848308860215328,
 'ConfidenceInterval': [-0.01536441741837822, 0.38572981522202643]}

In [21]:
vars(r2.dHtest)

{'TestStat': None,
 'criticalVal': 2.0728710884640007,
 'ConfidenceInterval': [-0.011967543025953466, 0.3823329408296017]}

In [22]:
r3=setBLP.oneDproj(yl,yu,x,0)

In [23]:
vars(r3)

{'bound': [-0.009051777215651783, 0.1443238930093685],
 'Htest': <setBLP.TestResults at 0x150c9aad0>,
 'dHtest': <setBLP.TestResults at 0x150d0f850>}

In [24]:
vars(r3.Htest)

{'TestStat': None,
 'criticalVal': 1.2750807017799315,
 'ConfidenceInterval': [-0.017511990312100045, 0.15278410610581675]}

In [25]:
vars(r3.dHtest)

{'TestStat': None,
 'criticalVal': 0.9542935369205432,
 'ConfidenceInterval': [-0.015383554229954473, 0.15065567002367117]}

In [26]:
r4=setBLP.oneDproj(yl,yu,x,1)

In [27]:
vars(r4)

{'bound': [-0.025567837808788303, 0.34563822843043474],
 'Htest': <setBLP.TestResults at 0x150d30f90>,
 'dHtest': <setBLP.TestResults at 0x150d045d0>}

In [28]:
vars(r4.Htest)

{'TestStat': None,
 'criticalVal': 5.647358010109039,
 'ConfidenceInterval': [-0.06303829135009092, 0.38310868197173736]}

In [29]:
vars(r4.dHtest)

{'TestStat': None,
 'criticalVal': 5.249938514985009,
 'ConfidenceInterval': [-0.06040139659296797, 0.3804717872146144]}

In [30]:
r5=setBLP.oneDproj(yl,yu,x)

In [31]:
vars(r5[0])

{'bound': [-0.009051777215651783, 0.1443238930093685],
 'Htest': <setBLP.TestResults at 0x150a14410>,
 'dHtest': <setBLP.TestResults at 0x150d0b950>}

In [32]:
vars(r5[0].Htest)

{'TestStat': None,
 'criticalVal': 0.7057800390978024,
 'ConfidenceInterval': [-0.013734656912326502, -0.020884958112113584]}

In [33]:
vars(r5[0].dHtest)

{'TestStat': None,
 'criticalVal': 0.6446082859263649,
 'ConfidenceInterval': [-0.013328779804633427, -0.02129083521980666]}

In [34]:
vars(r5[1])

{'bound': [-0.025567837808788303, 0.34563822843043474],
 'Htest': <setBLP.TestResults at 0x150d0b790>,
 'dHtest': <setBLP.TestResults at 0x15091ecd0>}

In [35]:
vars(r5[1].Htest)

{'TestStat': None,
 'criticalVal': 2.4007462549571277,
 'ConfidenceInterval': [0.12839484249587174, 0.3615672789439315]}

In [36]:
vars(r5[1].dHtest)

{'TestStat': None,
 'criticalVal': 2.026688153335207,
 'ConfidenceInterval': [0.13087673344266182, 0.3590853879971414]}

In [37]:
r6 = setBLP.oneDproj(yl, 'logwage_u', 'age', data = cps)

In [38]:
vars(r6)

{'bound': [-0.004887858329798024, 0.14865963017399167],
 'Htest': <setBLP.TestResults at 0x150ca02d0>,
 'dHtest': <setBLP.TestResults at 0x150d89210>}

In [39]:
vars(r6.Htest)

{'TestStat': None,
 'criticalVal': 0.7334538820404218,
 'ConfidenceInterval': [-0.00975435511687759, 0.15352612696107124]}

In [40]:
vars(r6.dHtest)

{'TestStat': None,
 'criticalVal': 0.6328099760557074,
 'ConfidenceInterval': [-0.009086578645709144, 0.1528583504899028]}

In [41]:
r7 = setBLP.oneDproj(yl, yu, 'educ', data = cps)

In [42]:
vars(r7)

{'bound': [0.0017860422219384202, 0.3685793555817098],
 'Htest': <setBLP.TestResults at 0x150d13110>,
 'dHtest': <setBLP.TestResults at 0x150d5a610>}

In [43]:
vars(r7.Htest)

{'TestStat': None,
 'criticalVal': 2.3752603908014684,
 'ConfidenceInterval': [-0.013973908530732972, 0.3843393063343812]}

In [44]:
vars(r7.dHtest)

{'TestStat': None,
 'criticalVal': 1.9509430348851013,
 'ConfidenceInterval': [-0.011158545356807749, 0.38152394316045596]}

In [45]:
r8 = setBLP.oneDproj(yl, yu, ['age','educ'], 0, data = cps)

In [46]:
vars(r8)

{'bound': [-0.009051777215651783, 0.1443238930093685],
 'Htest': <setBLP.TestResults at 0x150d81ed0>,
 'dHtest': <setBLP.TestResults at 0x150d88910>}

In [47]:
vars(r8.Htest)

{'TestStat': None,
 'criticalVal': 1.293213530107258,
 'ConfidenceInterval': [-0.017632302376643372, 0.15290441817036007]}

In [48]:
vars(r8.dHtest)

{'TestStat': None,
 'criticalVal': 0.9506623920091679,
 'ConfidenceInterval': [-0.015359461433560146, 0.15063157722727685]}

In [49]:
r9 = setBLP.oneDproj('logwage_l', 'logwage_u', ['age','educ'], 1, data = cps)

In [50]:
vars(r9)

{'bound': [-0.025567837808788303, 0.34563822843043474],
 'Htest': <setBLP.TestResults at 0x150d82c50>,
 'dHtest': <setBLP.TestResults at 0x150daa090>}

In [51]:
vars(r9.Htest)

{'TestStat': None,
 'criticalVal': 5.681036878980088,
 'ConfidenceInterval': [-0.06326175203544102, 0.3833321426570875]}

In [52]:
vars(r9.dHtest)

{'TestStat': None,
 'criticalVal': 5.238847388745805,
 'ConfidenceInterval': [-0.06032780651248735, 0.3803981971341338]}

In [53]:
r10 = setBLP.oneDproj(yl, yu, ['age','educ'], data = cps)

In [56]:
vars(r10[0])

{'bound': [-0.009051777215651783, 0.1443238930093685],
 'Htest': <setBLP.TestResults at 0x150d0ad50>,
 'dHtest': <setBLP.TestResults at 0x150d09f50>}

In [57]:
vars(r10[0].Htest)

{'TestStat': None,
 'criticalVal': 0.7049908492855393,
 'ConfidenceInterval': [-0.013729420605341406, -0.02089019441909868]}

In [58]:
vars(r10[0].dHtest)

{'TestStat': None,
 'criticalVal': 0.5907597488937556,
 'ConfidenceInterval': [-0.012971492538351446, -0.02164812248608864]}

In [59]:
vars(r10[1])

{'bound': [-0.025567837808788303, 0.34563822843043474],
 'Htest': <setBLP.TestResults at 0x150d82a90>,
 'dHtest': <setBLP.TestResults at 0x150cb9790>}

In [60]:
vars(r10[1].Htest)

{'TestStat': None,
 'criticalVal': 2.4391807997844244,
 'ConfidenceInterval': [0.12813982770421972, 0.3618222937355835]}

In [61]:
vars(r10[1].dHtest)

{'TestStat': None,
 'criticalVal': 1.9991493046954785,
 'ConfidenceInterval': [0.1310594548402939, 0.3589026665995093]}

# Testing CI1d  

$H_0: \beta_{age} = [0, 0.15]$

In [62]:
I1 = [0, 0.15]

In [63]:
result1 = setBLP.oneDproj(yl, yu, x1, H0=I1)

In [64]:
vars(result1)

{'bound': [-0.004887858329798024, 0.14865963017399167],
 'Htest': <setBLP.TestResults at 0x150d59990>,
 'dHtest': <setBLP.TestResults at 0x150d59f90>}

In [65]:
vars(result1.Htest)

{'TestStat': 0.7366733861557483,
 'criticalVal': 0.7139171525114368,
 'ConfidenceInterval': [-0.009624728109902926, 0.15339649995409657]}

In [66]:
vars(result1.dHtest)

{'TestStat': 0.20201378841259215,
 'criticalVal': 0.6171743707559667,
 'ConfidenceInterval': [-0.008982835759048631, 0.15275460760324228]}

$H_0: \beta_{age} = [0.1, 0.25]$

In [67]:
I2 = [0.1, 0.25]

In [68]:
result2 = setBLP.oneDproj(yl, yu, x1, H0=I2)

In [69]:
vars(result2)

{'bound': [-0.004887858329798024, 0.14865963017399167],
 'Htest': <setBLP.TestResults at 0x150d59a90>,
 'dHtest': <setBLP.TestResults at 0x150d0d650>}

In [70]:
vars(result2.Htest)

{'TestStat': 15.808169662239287,
 'criticalVal': 0.7188110546089332,
 'ConfidenceInterval': [-0.009657199352358543, 0.1534289711965522]}

In [71]:
vars(result2.dHtest)

{'TestStat': 15.273510064496133,
 'criticalVal': 0.6214583479800035,
 'ConfidenceInterval': [-0.009011260124788846, 0.1527830319689825]}

$H_0: \beta_{age} = [0, 0.1]$

In [72]:
I3 = [0, 0.1]

In [73]:
result3 = setBLP.oneDproj(yl, yu, x1, H0=I3)

In [74]:
vars(result3)

{'bound': [-0.004887858329798024, 0.14865963017399167],
 'Htest': <setBLP.TestResults at 0x15006acd0>,
 'dHtest': <setBLP.TestResults at 0x1060a0ad0>}

In [75]:
vars(result3.Htest)

{'TestStat': 7.333734349629176,
 'criticalVal': 0.7031486859138284,
 'ConfidenceInterval': [-0.009553278889464896, 0.15332505073365854]}

In [76]:
vars(result3.dHtest)

{'TestStat': 0.0,
 'criticalVal': 0.6053489212391338,
 'ConfidenceInterval': [-0.008904373413305307, 0.15267614525749895]}

$H_0: \beta_{age} = [-0.1, 0.15]$

In [77]:
I4 = [-0.1, 0.15]

In [78]:
result4 = setBLP.oneDproj(yl, yu, x1, H0=I4)

In [79]:
vars(result4)

{'bound': [-0.004887858329798024, 0.14865963017399167],
 'Htest': <setBLP.TestResults at 0x150d82bd0>,
 'dHtest': <setBLP.TestResults at 0x150d83090>}

In [80]:
vars(result4.Htest)

{'TestStat': 14.334822889927793,
 'criticalVal': 0.73837679954266,
 'ConfidenceInterval': [-0.009787018877741534, 0.15355879072193518]}

In [81]:
vars(result4.dHtest)

{'TestStat': 14.334822889927793,
 'criticalVal': 0.6158786865587252,
 'ConfidenceInterval': [-0.008974238840909206, 0.15274601068510285]}

# regress $yl, yu$ on age, educ $\rightarrow$ $\beta_{age}$

In [82]:
ols3 = smf.ols(formula='logwage_l ~ educ+age',data=cps).fit(cov_type='HC1')
betal = ols3._results.params[2]
print(ols3.summary())

                            OLS Regression Results                            
Dep. Variable:              logwage_l   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     982.3
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        12:13:02   Log-Likelihood:                -56435.
No. Observations:               22715   AIC:                         1.129e+05
Df Residuals:                   22712   BIC:                         1.129e+05
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.3251      0.151     21.988      0.0

In [83]:
ols4 = smf.ols(formula='logwage_u ~ educ+age',data=cps).fit(cov_type='HC1')
betau = ols4._results.params[2]
print(ols4.summary())

                            OLS Regression Results                            
Dep. Variable:              logwage_u   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     3220.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        12:13:03   Log-Likelihood:                -27984.
No. Observations:               22715   AIC:                         5.597e+04
Df Residuals:                   22712   BIC:                         5.600e+04
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.5141      0.037    201.564      0.0

In [84]:
I5 = [min(betal, betau), max(betal, betau)]

In [85]:
result5 = setBLP.oneDproj(yl, yu, x1, H0=I5)

In [86]:
vars(result5)

{'bound': [-0.004887858329798024, 0.14865963017399167],
 'Htest': <setBLP.TestResults at 0x150cb5810>,
 'dHtest': <setBLP.TestResults at 0x1519d1150>}

In [87]:
vars(result5.Htest)

{'TestStat': 7.556630260161291,
 'criticalVal': 0.6929737443891848,
 'ConfidenceInterval': [-0.00948576773238894, 0.15325753957658259]}

In [88]:
vars(result5.dHtest)

{'TestStat': 0.0,
 'criticalVal': 0.5946620129917048,
 'ConfidenceInterval': [-0.008833465335887754, 0.1526052371800814]}

# Two ways to access the list of all attributes and values:

In [ ]:
result.__dict__

In [ ]:
vars(result)

In [ ]:
vars(result.dHtest)

# Simulations

## parameter

In [ ]:
rng = np.random.MT19937(15217)
rng = np.random.Generator(rng)
np.random.seed(setBLP.default_options.seed)

In [ ]:
popSize = cps.shape[0]

In [ ]:
Nobs = 100;  #size of sub sample
Nsim = 5000; #number of simulations

In [ ]:
Nintervals = 8; #number of intervals in the survey

## Using quantiles

In [ ]:
c = 0 
width=0
for i in range(Nsim):
    indx = rng.integers(low=0, high=popSize, size=Nobs)
    sample = cps.iloc[indx, :]
    r = setBLP.oneDproj(sample.logwage_l,sample.logwage_u,sample.educ)
    width += max(r)-min(r)
    if beta1>=min(r) and beta1 <=max(r):
        c+=1
    
width=width/Nsim

In [ ]:
i = 0 
indx = rng.integers(low=0, high=popSize, size=Nobs)
sample = cps.iloc[indx, :]
r = setBLP.oneDproj(sample.logwage_l,sample.logwage_u,sample.educ)

In [ ]:
r

In [ ]:
c/Nsim

In [ ]:
width

## Using fixed intervals, ver 1

In [ ]:
thresholds = np.linspace(1,max(cps.wage)+1,10)
thresholds

In [ ]:
interval_cps = createIntervalData(cps, 'wage', thresholds)
interval_cps.head(15)

In [ ]:
c = 0 
width =0
for i in range(Nsim):
    indx = rng.integers(low=0, high=popSize, size=Nobs)
    sample = interval_cps.iloc[indx, :]
    r = setBLP.oneDproj(sample.logwage_l,sample.logwage_u,sample.educ)
    width+=max(r)-min(r)
    if beta1>=min(r) and beta1 <=max(r):
        c+=1
width=width/Nsim

In [ ]:
c/Nsim

In [ ]:
width

## Using fixed intervals, ver 2

In [ ]:
max(cps.wage)

In [ ]:
thresholds = np.array([0, 10, 20, 40, 75, 100, 200, 300, 500 ])*1000
thresholds[0] =1

In [ ]:
interval_cps = createIntervalData(cps, 'wage', thresholds)
interval_cps.head(15)

In [ ]:
c = 0 
width =0
for i in range(Nsim):
    indx = rng.integers(low=0, high=popSize, size=Nobs)
    sample = interval_cps.iloc[indx, :]
    r = setBLP.oneDproj(sample.logwage_l,sample.logwage_u,sample.educ)
    width+=max(r)-min(r)
    if beta1>=min(r) and beta1 <=max(r):
        c+=1
        
width=width/Nsim

In [ ]:
c/Nsim

In [ ]:
width

### Conclusion:  
The id interval is so wide that there is no chance that true $\beta_1$ is not in the computed identification set.

(age, wage) pair